In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import Voronoi
from scipy.stats import wilcoxon, ranksums
from shapely.geometry import Point, LineString
from shapely.ops import polygonize
from sklearn.cluster import KMeans
import numpy as np

In [4]:
# Load shp
VORONOI_P=gpd.read_file("/src/data/procesados/geo/poligonos_recortados.shp", encoding = 'utf-8')
VORONOI_L=gpd.read_file("/src/data/procesados/geo/lineas_voronoi.shp", encoding = 'utf-8')

In [5]:
VALID_W=pd.read_csv("/src/data/procesados/filtered/filterwords.csv", sep="\t", index_col=0, decimal=b',') 
VALID_W=VALID_W.transpose().sort_index().transpose()

In [6]:
def ecotone_mapping(voronoi_lines_frame):
    
    #all corpus frames
    all_pratt_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/normalized_allcorpus/pratt/nstat.csv", sep="\t", index_col=0, decimal=b',')
    all_pratt_pv_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/normalized_allcorpus/pratt/pvalue.csv", sep="\t", index_col=0, decimal=b',')
    all_wilcox_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/normalized_allcorpus/wilcox/nstat.csv", sep="\t", index_col=0, decimal=b',')
    all_wilcox_pv_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/normalized_allcorpus/wilcox/pvalue.csv", sep="\t", index_col=0, decimal=b',')
    all_zsplit_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/normalized_allcorpus/zsplit/nstat.csv", sep="\t", index_col=0, decimal=b',')
    all_zsplit_pv_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/normalized_allcorpus/zsplit/pvalue.csv", sep="\t", index_col=0, decimal=b',')
    all_dist_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/normalized_allcorpus/pratt/distancia.csv", sep="\t", index_col=0, decimal=b',')

    #intersected corpus frames
    int_pratt_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/intersected_corpus/pratt/nstat.csv", sep="\t", index_col=0, decimal=b',')
    int_pratt_pv_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/intersected_corpus/pratt/pvalue.csv", sep="\t", index_col=0, decimal=b',')
    int_wilcox_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/intersected_corpus/wilcox/nstat.csv", sep="\t", index_col=0, decimal=b',')
    int_wilcox_pv_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/intersected_corpus/wilcox/pvalue.csv", sep="\t", index_col=0, decimal=b',')
    int_zsplit_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/intersected_corpus/zsplit/nstat.csv", sep="\t", index_col=0, decimal=b',')
    int_zsplit_pv_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/intersected_corpus/zsplit/pvalue.csv", sep="\t", index_col=0, decimal=b',')
    int_dist_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/intersected_corpus/pratt/distancia.csv", sep="\t", index_col=0, decimal=b',')

    #samllest corpus frames
    small_pratt_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/smallest_corpus/pratt/nstat.csv", sep="\t", index_col=0, decimal=b',')
    small_pratt_pv_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/smallest_corpus/pratt/pvalue.csv", sep="\t", index_col=0, decimal=b',')
    small_wilcox_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/smallest_corpus/wilcox/nstat.csv", sep="\t", index_col=0, decimal=b',')
    small_wilcox_pv_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/smallest_corpus/wilcox/pvalue.csv", sep="\t", index_col=0, decimal=b',')
    small_zsplit_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/smallest_corpus/zsplit/nstat.csv", sep="\t", index_col=0, decimal=b',')
    small_zsplit_pv_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/smallest_corpus/zsplit/pvalue.csv", sep="\t", index_col=0, decimal=b',')
    small_dist_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/smallest_corpus/pratt/distancia.csv", sep="\t", index_col=0, decimal=b',')
        
    # QUITAR VALORES no probados
        
    # para all corpus pratt
    valid_pval=all_pratt_pv_frame.iloc[0:].mask(all_pratt_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    all_pratt_frame = all_pratt_frame * valid_pval
        
    # para all corpus wilcox
    valid_pval=all_wilcox_pv_frame.iloc[0:].mask(all_wilcox_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    all_wilcox_frame = all_wilcox_frame * valid_pval
        
    # para all corpus zsplit
    valid_pval=all_zsplit_pv_frame.iloc[0:].mask(all_zsplit_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    all_zsplit_frame = all_zsplit_frame * valid_pval
        
    # para all corpus dist
    valid_pval=all_pratt_pv_frame.iloc[0:].mask(all_pratt_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    all_dist_frame = all_dist_frame * valid_pval
              
    #intersected corpus pratt
    valid_pval=int_pratt_pv_frame.iloc[0:].mask(int_pratt_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    int_pratt_frame = int_pratt_frame * valid_pval
        
    #intersected corpus wilcox
    valid_pval=int_wilcox_pv_frame.iloc[0:].mask(int_wilcox_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    int_wilcox_frame = int_wilcox_frame * valid_pval

    #intersected corpus zsplit
    valid_pval=int_zsplit_pv_frame.iloc[0:].mask(int_zsplit_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    int_zsplit_frame = int_zsplit_frame * valid_pval

    #intersected corpus dist
    valid_pval=int_pratt_pv_frame.iloc[0:].mask(int_pratt_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    int_dist_frame = int_dist_frame * valid_pval
        
    #samllest corpus pratt
    valid_pval=small_pratt_pv_frame.iloc[0:].mask(small_pratt_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    small_pratt_frame = small_pratt_frame * valid_pval

    #samllest corpus wilcox
    valid_pval=small_wilcox_pv_frame.iloc[0:].mask(small_wilcox_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    small_wilcox_frame  = small_wilcox_frame  * valid_pval
        
    #samllest corpus zsplit
    valid_pval=small_zsplit_pv_frame.iloc[0:].mask(small_zsplit_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    small_zsplit_frame  = small_zsplit_frame  * valid_pval
        
    #samllest corpus dist
    valid_pval=small_wilcox_pv_frame.iloc[0:].mask(small_wilcox_pv_frame.iloc[0:]>=0.05,np.nan)
    valid_pval=valid_pval.iloc[0:].mask(valid_pval.iloc[0:]<0.05,1)
    small_dist_frame  = small_dist_frame  * valid_pval


    for line in voronoi_lines_frame.iterrows():
        
        ciudad1=line[1]["Ciudad1"]
        ciudad2=line[1]["Ciudad2"]

        # All
        voronoi_lines_frame.loc[(line[0]),"AllPrattW"]=float(all_pratt_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"AllPrattPV"]=float(all_pratt_pv_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"AllWilcoxW"]=float(all_wilcox_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"AllWilcoxP"]=float(all_wilcox_pv_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"AllZsplitW"]=float(all_zsplit_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"AllZsplitP"]=float(all_zsplit_pv_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"AllDist"]=float(all_dist_frame.loc[ciudad1,ciudad2])
        
        # intersected
        voronoi_lines_frame.loc[(line[0]),"IntPrattW"]=float(int_pratt_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"IntPrattP"]=float(int_pratt_pv_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"IntWilcoxW"]=float(int_wilcox_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"IntWilcoxP"]=float(int_wilcox_pv_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"IntZsplitW"]=float(int_zsplit_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"IntZsplitP"]=float(int_zsplit_pv_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"IntDist"]=float(int_dist_frame.loc[ciudad1,ciudad2])
        
        # small
        voronoi_lines_frame.loc[(line[0]),"SmPrattW"]=float(small_pratt_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"SmPrattP"]=float(small_pratt_pv_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"SmWilcoxW"]=float(small_wilcox_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"SmWilcoxP"]=float(small_wilcox_pv_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"SmZsplitW"]=float(small_zsplit_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"SmZsplitP"]=float(small_zsplit_pv_frame.loc[ciudad1,ciudad2])
        voronoi_lines_frame.loc[(line[0]),"SmDist"]=float(small_dist_frame.loc[ciudad1,ciudad2])
        
    vorolines=voronoi_lines_frame.drop(labels="cat",axis=1)
    vorolines.to_file("/src/data/procesados/geo/Ecotones/todos.shp", 
                                encoding = 'utf-8',
                                driver = 'ESRI Shapefile')
   
    
ecotone_mapping(VORONOI_L)

In [24]:
# Load shp
VORONOI_P=gpd.read_file("/src/data/procesados/geo/poligonos_recortados.shp", encoding = 'utf-8')
VORONOI_L=gpd.read_file("/src/data/procesados/geo/lineas_voronoi.shp", encoding = 'utf-8')
def coroplet_mapping(voronoi_poligons_frame, destino):
    
    coroplets = voronoi_poligons_frame
    #all corpus frames
    all_pratt_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/normalized_allcorpus/pratt/nstat.csv", sep="\t", index_col=0, decimal=b',')
    all_pratt_frame.drop(labels="#TOTALES_COLOMBIA#", inplace=True)
    all_pratt_frame.drop(labels="#TOTALES_COLOMBIA#", axis=1, inplace=True)
    
    #intersected corpus frames
    int_pratt_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/intersected_corpus/pratt/nstat.csv", sep="\t", index_col=0, decimal=b',')
    int_pratt_frame.drop(labels="#TOTALES_COLOMBIA#", inplace=True)
    int_pratt_frame.drop(labels="#TOTALES_COLOMBIA#", axis=1, inplace=True)
    
    #samllest corpus frames
    small_pratt_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/smallest_corpus/pratt/nstat.csv", sep="\t", index_col=0, decimal=b',')
    small_pratt_frame.drop(labels="#TOTALES_COLOMBIA#", inplace=True)
    small_pratt_frame.drop(labels="#TOTALES_COLOMBIA#", axis=1, inplace=True)


    coroplets.set_index("ciudad", inplace=True)
    coroplets.sort_index(inplace=True)
    
  
    #all pratt
    all_pratt_frame.sort_index(inplace=True)
    if (np.all(all_pratt_frame.index==coroplets.index)):
        allpratt2 = KMeans(n_clusters=2, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(all_pratt_frame.as_matrix())
        allpratt3 = KMeans(n_clusters=3, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(all_pratt_frame.as_matrix())
        allpratt4 = KMeans(n_clusters=4, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(all_pratt_frame.as_matrix())
        allpratt5 = KMeans(n_clusters=5, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(all_pratt_frame.as_matrix())
        allpratt6 = KMeans(n_clusters=6, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(all_pratt_frame.as_matrix())
        allpratt7 = KMeans(n_clusters=7, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(all_pratt_frame.as_matrix())      
 
    #int pratt
    int_pratt_frame.sort_index(inplace=True)
    if (np.all(int_pratt_frame.index==coroplets.index)):
        intpratt2 = KMeans(n_clusters=2, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(int_pratt_frame.as_matrix())
        intpratt3 = KMeans(n_clusters=3, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(int_pratt_frame.as_matrix())
        intpratt4 = KMeans(n_clusters=4, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(int_pratt_frame.as_matrix())
        intpratt5 = KMeans(n_clusters=5, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(int_pratt_frame.as_matrix())
        intpratt6 = KMeans(n_clusters=6, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(int_pratt_frame.as_matrix())
        intpratt7 = KMeans(n_clusters=7, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(int_pratt_frame.as_matrix())
            
    #small pratt
    small_pratt_frame.sort_index(inplace=True)
    if (np.all(small_pratt_frame.index==coroplets.index)):
        smallpratt2 = KMeans(n_clusters=2, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(small_pratt_frame.as_matrix())
        smallpratt3 = KMeans(n_clusters=3, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(small_pratt_frame.as_matrix())
        smallpratt4 = KMeans(n_clusters=4, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(small_pratt_frame.as_matrix())
        smallpratt5 = KMeans(n_clusters=5, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(small_pratt_frame.as_matrix())
        smallpratt6 = KMeans(n_clusters=6, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(small_pratt_frame.as_matrix())
        smallpratt7 = KMeans(n_clusters=7, n_init=100, max_iter=500,tol=1e-7,n_jobs=6).fit(small_pratt_frame.as_matrix())
    
    #2    
    coroplets["AllW2"]=allpratt2.labels_
    coroplets["INTW2"]=intpratt2.labels_
    coroplets["SMAW2"]=smallpratt2.labels_
    
    #3
    coroplets["AllW3"]=allpratt3.labels_
    coroplets["INTW3"]=intpratt3.labels_
    coroplets["SMAW3"]=smallpratt3.labels_
    
    #4
    coroplets["AllW4"]=allpratt4.labels_
    coroplets["INTW4"]=intpratt4.labels_
    coroplets["SMAW4"]=smallpratt4.labels_
    
    #5
    coroplets["AllW5"]=allpratt5.labels_
    coroplets["INTW5"]=intpratt5.labels_
    coroplets["SMAW5"]=smallpratt5.labels_
    
    #6
   
    coroplets["AllW6"]=allpratt6.labels_
    coroplets["INTW6"]=intpratt6.labels_
    coroplets["SMAW6"]=smallpratt6.labels_
    
    #7
   
    coroplets["AllW7"]=allpratt7.labels_
    coroplets["INTW7"]=intpratt7.labels_
    coroplets["SMAW7"]=smallpratt7.labels_ 
    
    coroplets.reset_index(inplace=True)
    coroplets.to_file("/src/data/procesados/geo/coropletas/{}".format(destino),
                                   encoding = 'utf-8',
                                   driver = 'ESRI Shapefile')
    return(coroplets)

In [40]:
#coroplet_mapping(VORONOI_P,"agrupaciones.shp")

In [143]:
# Load shp
VORONOI_P=gpd.read_file("/src/data/procesados/geo/poligonos_recortados.shp", encoding = 'utf-8')

def coroplet_distance_to_city(voronoi_poligons_frame, destino):
    coroplets = voronoi_poligons_frame
    
    #samllest corpus frames
    small_wilcox_frame = pd.read_csv("/src/data/procesados/wilcoxon_signed/smallest_corpus/pratt/nstat.csv", sep="\t", index_col=0, decimal=b',')
    small_wilcox_frame.drop(labels="#TOTALES_COLOMBIA#", inplace=True)
    #small_wilcox_frame.drop(labels="#TOTALES_COLOMBIA#", axis=1,inplace=True)
    
    coroplets.set_index("ciudad", inplace=True)
    coroplets.sort_index(inplace=True)
    coroplets=coroplets.join(small_wilcox_frame.iloc[0:,0:31])
    #return(small_wilcox_frame.iloc[0:,32])
    coroplets.to_file("/src/data/procesados/geo/coropletas/{}".format(destino),
                    encoding = 'utf-8',
                    driver = 'ESRI Shapefile')
    return(coroplets)

In [144]:
coroplet_distance_to_city(VORONOI_P, "dist.shp")

,geometry,#TOTALES_COLOMBIA#,Acevedo,Aguachica,Agustín Codazi,Albania,Andes,Apartadó,Arauca,Armenia,...,Cali,Campo de la Cruz,Cáqueza,Carepa,Cartagena,Caucasia,Cereté,Chaparral,Chigorodó,Chimichagua
ciudad,,,,,,,,,,,,,,,,,,,,,
Acevedo,POLYGON ((-76.06601820636112 1.230556350496693...,0.258741,1.000000,0.095127,0.260766,0.230999,0.279123,0.221300,0.057281,0.362800,...,0.239129,0.169774,0.185240,0.362106,0.266907,0.201183,0.293172,0.347232,0.329704,0.288755
Aguachica,POLYGON ((-73.64494873133563 7.667910844395635...,0.085317,0.095127,1.000000,0.296623,0.285338,0.328239,0.157566,0.144438,0.255046,...,0.090824,0.217955,0.139352,0.369572,0.096687,0.260156,0.147329,0.389437,0.354684,0.310139
Agustín Codazi,POLYGON ((-72.92667474611501 10.22886322338355...,0.281987,0.260766,0.296623,1.000000,0.303826,0.287299,0.267480,0.305635,0.363239,...,0.270423,0.282241,0.294783,0.356974,0.276997,0.253380,0.288972,0.392709,0.358220,0.275803
Albania,POLYGON ((-72.36170469037926 11.16123305742052...,0.229297,0.230999,0.285338,0.303826,1.000000,0.322438,0.253724,0.244857,0.332685,...,0.226656,0.266497,0.294210,0.375358,0.213546,0.203467,0.243838,0.389273,0.352381,0.331735
Andes,POLYGON ((-75.95026646404057 5.426429077568432...,0.284604,0.279123,0.328239,0.287299,0.322438,1.000000,0.255980,0.301255,0.357695,...,0.282005,0.359989,0.297687,0.362289,0.298407,0.281058,0.312779,0.390559,0.332223,0.350453
Apartadó,POLYGON ((-76.87953291411674 7.911724661788894...,0.186875,0.221300,0.157566,0.267480,0.253724,0.255980,1.000000,0.257762,0.304468,...,0.167107,0.215739,0.262275,0.331085,0.183805,0.239270,0.223278,0.378862,0.341049,0.305520
Arauca,POLYGON ((-71.63914819376791 7.060547264712202...,0.151461,0.057281,0.144438,0.305635,0.244857,0.301255,0.257762,1.000000,0.299297,...,0.151713,0.201356,0.222588,0.359023,0.171175,0.247896,0.193068,0.372863,0.335665,0.311809
Armenia,POLYGON ((-75.73137816189305 4.094791062199091...,0.297149,0.362800,0.255046,0.363239,0.332685,0.357695,0.304468,0.299297,1.000000,...,0.254298,0.303762,0.305778,0.406526,0.188227,0.321892,0.223377,0.416939,0.405916,0.380306
Baranoa,POLYGON ((-75.15799681296114 10.84009859108857...,0.159519,0.358483,0.231391,0.324740,0.276837,0.335443,0.259249,0.263035,0.053132,...,0.130628,0.249194,0.291559,0.371554,0.118680,0.271612,0.190909,0.396918,0.369708,0.339182
